In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在GCP上进行E2E ML：从Vertex AI Feature Store开始服务

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_vertex_feature_store_serving.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
    
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_vertex_feature_store_serving.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
        </a>
  </td>
    
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_vertex_feature_store_serving.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
    
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在生产环境中使用Google Cloud上的Vertex AI进行端到端的MLOps。本教程涵盖了第3阶段：提供服务：从特征存储开始使用服务。

### 目标

在本教程中，您将学习如何使用`Vertex AI Feature Store`来训练模型，以及在进行在线和批量预测时提供特征。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI Feature Store`
- `Vertex AI Training`
- `Vertex AI Prediction`

执行的步骤包括：

- 创建一个 Vertex AI `Featurestore` 资源。
    - 为 `Featurestore` 资源创建 `EntityType` 资源。
    - 为每个 `EntityType` 资源创建 `Feature` 资源。
- 将特征值（实体数据项）导入到 `Featurestore` 资源中。
    - 来自 Cloud Storage 位置。
    - 来自 pandas DataFrame。
- 从 `Featurestore` 资源执行在线预测。
- 从 `Featurestore` 资源执行批量预测。

### 数据集

本笔记本中使用的数据集包含自2018年以来的在线电子商务商店订单数据。此数据集可以在`bigquery-public-data.thelook_ecommerce.order_items`的BigQuery表中公开获取，可以通过在BigQuery中固定bigquery-public-data项目来访问。

该表包含与每个订单项有关的各种字段，如订单ID、产品ID、用户ID、状态以及创建时的价格，发货时的价格等。在这些字段中，当前笔记本使用以下字段，假定它们的用途如下描述：

* user_id：用户的ID。
* product_id：产品的ID。
* created_at：用户下订单的时间。
* status：订单的状态（已发货、处理中、取消、退货和已完成）。

该数据集用于训练一个推荐模型。

### 费用
本教程使用 Google Cloud 的计费组件：

- Vertex AI
- Cloud Storage
- BigQuery

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)、[Cloud Storage 价格](https://cloud.google.com/storage/pricing) 和[BigQuery 价格](https://cloud.google.com/bigquery/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成费用估算。

## 安装

安装以下软件包以进一步运行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install the dependecies
! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-bigquery \
                         pyarrow \
                         pandas {USER_FLAG} --quiet

重新启动内核

安装了额外的软件包后，需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### 设置您的Google Cloud项目

**无论您使用什么笔记本环境，都需要按照以下步骤进行操作。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。首次创建帐户时，您将获得300美元的免费信用额，用于支付计算/存储成本。

1. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI、计算引擎、Cloud Storage和Cloud Logging API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage_component,logging)。

1. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK为本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter会将以`!`开头的行视为shell命令，并将以`$`前缀的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以通过下面的 `gcloud` 命令获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

在当前环境中设置默认项目ID

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您也可以更改“REGION”变量，该变量用于本笔记本的其余部分操作。以下是Vertex AI支持的地区。我们建议您选择最靠近您的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法在Vertex AI上使用多地区存储桶进行训练。并非所有地区都支持所有Vertex AI服务。

了解更多关于[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您正在进行实时教程会话，则可能正在使用共享的测试帐户或项目。为了避免在创建的资源之间发生名称冲突，您为每个实例会话创建一个 uuid，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，则您的环境已经得到验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行认证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

1. **单击创建服务帐户**。

2. 在**服务帐户名称**字段中输入名称，然后单击**创建**。

3. 在**授予此服务帐户访问项目**部分，单击角色下拉列表。在过滤框中输入"Vertex"，并选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，并选择**Storage对象管理员**。

4. 单击创建。包含您密钥的JSON文件将下载到本地环境中。

5. 在下面的单元格中将您的服务账户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# IMPORTANT - If you are using Vertex AI Workbench Notebooks, your environment is already authenticated. Skip this step.

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

当您初始化 Python 版 Vertex AI SDK 时，您需要指定一个云存储中转桶。这个中转桶是您的数据集和模型资源在会话之间保留的位置。

在下面设置您的云存储桶的名称。桶的名称必须在所有谷歌云项目中全局唯一，甚至包括您所在组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "vai-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查存储桶的内容来验证访问您的云存储桶。

In [ ]:
! gsutil ls -al $BUCKET_URI

为特征存储设置桶访问权限

In [ ]:
! gsutil uniformbucketlevelaccess set on {BUCKET_URI}

导入库并定义常量

In [ ]:
from datetime import datetime, timedelta

import google.cloud.aiplatform as aiplatform
import pandas as pd
from google.cloud import bigquery

初始化顶点 AI 和 BigQuery 客户端

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)
bqclient = bigquery.Client(project=PROJECT_ID)

设置预构建的容器

设置用于训练和预测的预构建Docker容器映像。


有关最新列表，请参见[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。


有关最新列表，请参见[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TF = "2.8".replace(".", "-")
TRAIN_VERSION = "tf-cpu.{}".format(TF)
DEPLOY_VERSION = "tf2-cpu.{}".format(TF)

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Training:", TRAIN_IMAGE)
print("Deployment:", DEPLOY_IMAGE)

#### 设置机器类型

接下来，设置用于训练的机器类型。

- 设置变量`TRAIN_COMPUTE`/`DEPLOY_COMPUTE` 来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个虚拟 CPU 3.75GB 的内存。
     - `n1-highmem`：每个虚拟 CPU 6.5GB 的内存。
     - `n1-highcpu`：每个虚拟 CPU 0.9GB 的内存。
 - `虚拟 CPU`：数量为 \[2, 4, 8, 16, 32, 64, 96 \]

*注意：以下机型不支持用于训练：*

 - `standard`：2 个虚拟 CPU
 - `highcpu`：2、4 和 8 个虚拟 CPU

*注意：您也可以使用 n2 和 e2 机型进行训练和部署，但它们不支持 GPU。*

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## 引言：Vertex AI 功能存储介绍

假设您有一个推荐模型，用于预测要在收银机收据背面打印的优惠券。现在，如果该模型只是在单个交易实例（购买了什么和多少）上进行训练，那么（过去）您会使用Apriori算法。

但现在我们有关于客户的历史数据（比如按信用卡号索引）。比如迄今为止的总购买金额，每笔交易的平均购买金额，按产品类别购买的频率等。我们使用这些“丰富的数据”来训练一个推荐系统。

现在是进行实时预测的时候了。您从收银机得到一笔交易，但它只有信用卡号和这笔交易。它没有模型所需的丰富数据。在提供服务的过程中，信用卡号被用作特征存储中获取模型所需的丰富数据的索引。

另一方面，假设模型训练时使用的丰富数据是时间戳为6月1日的。当前交易发生在6月15日。假设用户在6月1日至15日之间进行了其他交易，并且丰富的数据在特征存储中持续更新。但模型是基于6月1日的数据进行训练的。特征存储知道版本号，并将6月1日的版本提供给模型（而不是当前的6月15日）。否则，如果使用6月15日的数据，就会出现训练和提供服务之间的偏差。

这里的另一个问题是数据漂移。事情会发生变化，突然某一天，每个人都在购买卫生纸！现有的丰富数据的分布与部署模型训练时的分布发生了显著变化。特征存储可以检测到分布变化/阈值的变化，并触发重新训练模型的通知。

了解更多关于[Vertex AI 功能存储 API](https://cloud.google.com/vertex-ai/docs/featurestore)。

## Vertex AI 特征存储数据模型

Vertex AI 特征存储使用以下三个重要的层次概念组织数据：

        Featurestore（特征存储） -> EntityType（实体类型） -> Feature（特征）

- `Featurestore`：存储特征的地方。
- `EntityType`：在一个 `Featurestore` 下，`EntityType` 描述了要被建模的对象，可以是真实的也可以是虚拟的。
- `Feature`：在一个 `EntityType` 下，`Feature` 描述了 `EntityType` 的属性。

了解有关[Vertex AI 特征存储数据模型](https://cloud.google.com/vertex-ai/docs/featurestore/concepts)的更多信息。

在这个电子商务示例中，您将创建一个名为 ecomm_recommendation 的 `Featurestore` 资源。这个 `Featurestore` 资源有两个实体类型：
- `users`：这个实体类型具有 `product_id` 和 `rating` 特征。
- `products`：这个实体类型具有 `user_list` 和 `product_name` 特征。

## 创建 `Featurestore` 资源

首先，使用以下参数使用 `Featurestore.create()` 方法为数据集创建一个 `Featurestore`：

- `featurestore_id`：特征存储的名称。
- `online_store_fixed_node_count`：特征存储中在线服务的配置设置。
- `project`：项目 ID。
- `location`：位置（区域）。

In [ ]:
# Represents featurestore resource path.
FEATURESTORE_NAME = "ecomm_recommendation" + UUID

featurestore = aiplatform.Featurestore.create(
    featurestore_id=FEATURESTORE_NAME,
    online_store_fixed_node_count=1,
    project=PROJECT_ID,
    location=REGION,
)

print(featurestore)

获取“Featurestore”资源

您可以使用“Featurestore（）”初始化程序获取您项目中指定的“Featurestore”资源，具有以下参数：

- `featurestore_name`：`Featurestore`资源的名称。
- `project`：项目ID。
- `location`：位置（区域）。

In [ ]:
featurestore = aiplatform.Featurestore(
    featurestore_name=FEATURESTORE_NAME, project=PROJECT_ID, location=REGION
)
print(featurestore)

为您的`Featurestore`资源创建实体类型

接下来，您可以使用`create_entity_type()`方法为您的`Featurestore`资源创建`EntityType`资源，使用以下参数：

- `entity_type_id`：`EntityType`资源的名称。
- `description`：实体类型的描述。

In [ ]:
for name, description in [
    ("users", "Description of the user"),
    ("products", "Description of the product"),
]:
    entity_type = featurestore.create_entity_type(
        entity_type_id=name, description=description
    )
    print(entity_type)

### 为您的`EntityType`资源添加`Feature`资源

接下来，您使用`create_feature()`方法为您的`Featurestore`资源中的每个`EntityType`资源创建`Feature`资源，参数如下：

- `feature_id`：`Feature`资源的名称。
- `description`：特征的描述。
- `value_type`：特征的数据类型。

In [ ]:
def create_features(featurestore_name, entity_name, features):
    entity_type = aiplatform.EntityType(
        entity_type_name=entity_name, featurestore_id=featurestore_name
    )

    for feature in features:
        feature = entity_type.create_feature(
            feature_id=feature[0], description=feature[1], value_type=feature[2]
        )
        print(feature)


create_features(
    FEATURESTORE_NAME,
    "users",
    [
        ("product_id", "product description", "INT64"),
        ("rating", "rating of the product", "DOUBLE"),
    ],
)

create_features(
    FEATURESTORE_NAME,
    "products",
    [
        ("users_list", "List of user ids who bought product", "STRING_ARRAY"),
    ],
)

## 对数据集执行特征工程

接下来，对公开的BigQuery数据集执行特征工程，然后将其导入特征存储。

### 将BigQuery数据集加载到数据框中

* 将数据从BigQuery加载到pandas数据框中。
* 选择要使用的列。
    - 用户ID
    - 产品ID
    - 创建时间
    - 状态

In [ ]:
query_string = """
SELECT
    CAST(user_id AS STRING) AS user_id,
    product_id,
    created_at,
    status
FROM
    `bigquery-public-data.thelook_ecommerce.order_items`
"""

df_bq_table = bqclient.query(query_string).result().to_dataframe()

print(df_bq_table.shape)
df_bq_table.head()

### 派生一个新的列评分

接下来，您添加一个用于评分的新列。由于评分是数值型的，您可以从现有的状态列中派生出它们，具体做法如下：

- 将状态字符串值映射到数值范围（0..4）。
- 将值归一化在0和1之间。

In [ ]:
# map the status to a rating
rating_map = {
    "Cancelled": 0,
    "Returned": 1,
    "Processing": 2,
    "Shipped": 3,
    "Complete": 4,
}

df_bq_table["rating"] = df_bq_table["status"].map(rating_map)
print(df_bq_table.head())

# Normalize the ratings
min_rating = min(df_bq_table["rating"])
max_rating = max(df_bq_table["rating"])

df_bq_table["rating"] = (
    df_bq_table["rating"]
    .apply(lambda x: (x - min_rating) / (max_rating - min_rating))
    .values
)
print(df_bq_table.head())

### 过滤数据集

接下来，将数据集过滤为仅包含上周购买产品的用户，并删除“状态”列。

In [ ]:
PAST_WEEK_DATE = datetime.now() - pd.to_timedelta("7day")

df_filtered = df_bq_table[
    (df_bq_table["created_at"] < PAST_WEEK_DATE.isoformat() + "Z")
].reset_index()

result = df_filtered.groupby(["product_id"])["user_id"].apply(list).to_dict()

df_prod_user_list = pd.DataFrame(result.items(), columns=["product_id", "users_list"])
df_prod_user_list["product_id"] = df_prod_user_list["product_id"].astype("string")
print(df_prod_user_list.head())

df_bq_table.drop("status", axis=1, inplace=True)

### 将预处理数据重新导入BigQuery

#### 为预处理数据创建目标表。

接下来，您需要创建一个BigQuery数据集，随后将在其中添加预处理数据的表。

In [ ]:
DESTINATION_DATASET = f"product_recommendation_{UUID}"

USERS_SOURCE_TABLE_NAME = "user_prod_rating_data"
USERS_SOURCE_TABLE_URI = (
    f"bq://{PROJECT_ID}.{DESTINATION_DATASET}.{USERS_SOURCE_TABLE_NAME}"
)

PRODUCTS_SOURCE_TABLE_NAME = "prod_users_list_data"
PRODUCTS_SOURCE_TABLE_URI = (
    f"bq://{PROJECT_ID}.{DESTINATION_DATASET}.{PRODUCTS_SOURCE_TABLE_NAME}"
)

# Create destination dataset
dataset_id = "{}.{}".format(PROJECT_ID, DESTINATION_DATASET)
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION
dataset = bqclient.create_dataset(dataset)
print(dataset)

#### 为筛选后的数据集创建表格

接下来，您可以创建一个表格并加载筛选后的数据集。

In [ ]:
# Create a table
schema = [
    bigquery.SchemaField("user_id", "STRING"),
    bigquery.SchemaField("product_id", "INT64"),
    bigquery.SchemaField("created_at", "TIMESTAMP"),
    bigquery.SchemaField("rating", "FLOAT"),
]

table_id = f"{PROJECT_ID}.{DESTINATION_DATASET}.{USERS_SOURCE_TABLE_NAME}"
table = bigquery.Table(table_id, schema=schema)
bqclient.create_table(table, exists_ok=True)


# Load data to BQ
job = bqclient.load_table_from_dataframe(df_bq_table, table_id)
print(job.errors, job.state)
while job.running():
    from time import sleep

    sleep(30)
    print("Running ...")
print(job.errors, job.state)

创建用于产品用户列表的表格。

创建新的表格，用于产品用户列表。

In [ ]:
from time import sleep

# Create a table
schema = [
    bigquery.SchemaField("product_id", "STRING"),
    bigquery.SchemaField("users_list", "STRING", "REPEATED"),
]
table_id = f"{PROJECT_ID}.{DESTINATION_DATASET}.{PRODUCTS_SOURCE_TABLE_NAME}"
table = bigquery.Table(table_id, schema=schema)
bqclient.create_table(table, exists_ok=True)

# Load data to BQ
job = bqclient.load_table_from_dataframe(df_prod_user_list, table_id)
print(job.errors, job.state)
while job.running():
    sleep(30)
    print("Running ...")
print(job.errors, job.state)

## 将特征数据导入到您的`Featurestore`资源中

接下来，您将导入您的`Featurestore`资源的特征数据。一旦导入，您可以将这些特征值用于在线和离线（批处理）服务。

### 数据布局

每个导入的`EntityType`资源数据必须有一个ID。此外，每个`EntityType`资源数据项可以选择性地具有一个时间戳，指定特征值生成的时间。

在导入时，在您的请求中指定以下内容：

- 数据源格式：BigQuery表/Avro/CSV/Pandas数据框
- 数据源URL
- 目标：要导入的特征存储/实体类型/特征

在本教程中，模式如下：

    对于用户实体：
    模式 = {
        "name": "users",
        "fields": [
            {
                "name":"product_id",
                "type":["null","integer"]
            },
            {
                "name":"rating",
                "type":["null","double"]
                },
        ]
    }
    
    对于产品实体：
    模式 = {
        "name": "products",
        "fields": [
            {
                "name":"users_list",
                "type":["null","string_array"]
            }
        ]
    }


### 从BigQuery导入特征值

您可以使用`ingest_from_bq()`方法导入`EntityType`资源的特征值，参数如下：

- `entity_id_field`：父`EntityType`资源的标识符名称。
- `feature_ids`：要添加到`EntityType`资源中的`Feature`资源数据的标识符名称列表。
- `feature_time`：用于输入特征的时间戳字段。
- `bq_source_uri`：要从中导入数据的BigQuery表。

In [ ]:
entity_type = featurestore.get_entity_type("users")
response = entity_type.ingest_from_bq(
    entity_id_field="user_id",
    feature_ids=["product_id", "rating"],
    feature_time="created_at",
    bq_source_uri=f"bq://{PROJECT_ID}.{DESTINATION_DATASET}.{USERS_SOURCE_TABLE_NAME}",
)
print(response)


def past_6days():
    return datetime.now() - timedelta(days=6)


entity_type = featurestore.get_entity_type("products")
response = entity_type.ingest_from_bq(
    entity_id_field="product_id",
    feature_ids=["users_list"],
    feature_time=past_6days(),
    bq_source_uri=f"bq://{PROJECT_ID}.{DESTINATION_DATASET}.{PRODUCTS_SOURCE_TABLE_NAME}",
)
print(response)

顶点 AI 特征存储服务

顶点 AI 特征存储服务为从 `Featurestore` 资源中提供特征提供以下两种服务：

- 在线提供 - 低延迟提供小批量特征（预测）。

- 批量提供 - 高吞吐量提供大批量特征（训练和预测）。

批量服务

Vertex AI特征商店的批量服务功能专门针对实时高吞吐量大批量特征的服务优化，通常用于训练模型或批量预测。

可以将批量服务发送到以下目的地：

- BigQuery表
- 云存储位置
- 数据帧

### 输出数据集

在这个笔记本中，您将使用来自您的特征存储中以CSV格式存储在Google Cloud Storage中的数据来训练一个模型。

### 用例

**任务** 是准备一个数据集来训练一个模型，该模型为给定用户推荐产品。为了实现这一目标，您需要两组输入：

* 特征：您已经导入到特征存储中。
* 标签：记录在案的实际数据，即评分。

具体来说，实际观测数据在表1中描述，期望的数据集在表2中描述。表2中的每一行都是根据表1中的实体ID和时间戳连接导入的特征值从Vertex AI特征存储中获取的结果。在这个示例中，已选择从`users`中的`product_id`和`rating`特征进行批量训练。

batch_serve_to_df方法将表1作为read_instances_df参数的输入，连接所有需要的特征值从特征存储中，并返回用于训练的表2。

<h4 align="center">表1. 实际数据</h4>

users | timestamp            
----- | -------------------- 
87228 | 2022-07-01T00:00:00Z 
16173 | 2022-07-01T18:09:43Z 
...   | ...      | ...     


<h4 align="center">表2. batch_serve_to_df生成的预期训练数据（正样本）</h4>

feature_timestamp            | entity_type_users | product_id | rating |
-------------------- | ----------------- | --------------- | ---------------- |
2022-07-01T00:00:00Z | 87228 | 4567 | 0.5 |
2022-07-01T00:00:00Z | 16173 | 5490 | 0.75 |
... | ... | ... | ... | ...  

#### 为什么要有时间戳?

注意表2中有一个`timestamp`列。这表示观察到实际数据的时间。这是为了避免数据不一致。

例如，表2中的第一行表示ID为`87228`的用户在`2022-07-01T00:00:00Z`购买了产品。特征存储保留所有时间戳的特征值，但在批量服务期间仅获取给定时间戳时的特征值。

### 批量服务至数据框

组装请求，指定以下信息：

* 标签数据在哪里，即表1。
* 要读取哪些特征，即表1中的列名。

接下来，使用batch_serve_to_df从特征存储中获取数据框，并将其存储到一个CSV文件中，该文件将用于在Vertex AI中训练推荐模型。

* 将entityType Id（`users`）和`timestamp`列导出为csv到创建的GCS桶中。

In [ ]:
from datetime import timezone

past_week_date = (datetime.now() - pd.to_timedelta("7day")).isoformat() + "Z"
df_sorted = df_bq_table.sort_values("created_at", ascending=False, ignore_index=True)
df_sorted.rename(columns={"user_id": "users"}, inplace=True)
df_sorted = df_sorted[df_sorted["created_at"] <= past_week_date].reset_index()
df_sorted["created_at"] = df_sorted["created_at"].astype(str)
df_sorted["timestamp"] = df_sorted["created_at"].map(
    lambda x: datetime.fromisoformat(x).astimezone(timezone.utc)
)
df_batch = df_sorted[["users", "timestamp"]]

df_batch.head()

### 批量读取特征值

您可以使用`batch_serve_to_df`方法将实体数据项批量提供给DataFrame，参数如下：

- `serving_feature_ids`：要提供的实体类型和相应特征的字典。
- `read_instances_uri`：要从中读取实体数据项的云存储位置。

输出存储在一个BigQuery表中。

In [ ]:
batch_serve = featurestore.batch_serve_to_df(
    serving_feature_ids={"users": ["product_id", "rating"]}, read_instances_df=df_batch
)

batch_serve.head()

将数据框数据导出为CSV文件

接下来，您将数据框数据导出到云存储中的CSV文件。

In [ ]:
CSV_FILE = f"{BUCKET_URI}/data.csv"

batch_serve.to_csv(CSV_FILE, index=False)

## 训练一个推荐模型

在这个部分，您使用`batch_serve_to_df`方法的数据来训练一个为给定用户推荐产品的定制模型。

您可以使用Vertex AI SDK for Python在Docker容器中的Python脚本中创建一个定制训练的模型，然后通过发送数据获取部署模型的预测。

执行的步骤包括：

- 训练一个Vertex AI定制的`TrainingPipeline`来训练一个TensorFlow模型。
- 部署`Model`资源到服务`Endpoint`资源。
- 进行预测。

### 训练模型

您可以使用容器镜像有两种方式来训练模型：

- **使用 Vertex AI 预构建的容器**。如果您使用预构建的训练容器，您还必须指定一个要安装到容器镜像中的 Python 包。这个 Python 包包含您的训练代码。

- **使用自定义的容器镜像**。如果您使用自己的容器，容器镜像必须包含您的训练代码。

### 为训练脚本定义命令参数

准备要传递给训练脚本的命令行参数。
- `args`：要传递给相应 Python 模块的命令行参数。在这个例子中，它们是：
  - `"--epochs=" + EPOCHS`：用于训练的周期数。
  - `"--batch_size=" + BATCH_SIZE`：用于训练的批次大小。
  - `"--training_data=" + GCS_PATH`：来自特征存储的包含训练数据的 csv 文件的路径。

In [ ]:
EPOCHS = 20
BATCH_SIZE = 10

CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE),
    "--training_data=" + CSV_FILE,
]

#### 训练脚本

接下来，您将编写训练脚本 `task.py` 的内容。总结起来，该脚本执行以下操作：

- 从 Google Cloud 存储加载 csv 数据。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型（`compile()`）。
- 根据参数 `args.epochs` 和 `args.batch_size` 训练模型（`fit()`）。
- 从环境变量 `AIP_MODEL_DIR` 获取保存模型工件的目录。该变量由[训练服务设置](https://cloud.google.com/vertex-ai/docs/training/code-requirements#environment-variables)。
- 将训练好的模型保存到模型目录中。

In [ ]:
%%writefile task.py

import argparse
import tensorflow as tf
import numpy as np
import os

import pandas as pd


# Read args
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--batch_size', dest='batch_size',
                    default=10, type=int,
                    help='Batch size.')
parser.add_argument('--training_data', dest='training_data', type=str,
                    help="URI of the training data in BQ")

args = parser.parse_args()


# Collect the arguments
training_data_uri = args.training_data

# Set up training variables
LABEL_COLUMN = "rating"
UNUSED_COLUMNS = ["timestamp","entity_type_users","product_id"]
NA_VALUES = ["NA", ".", " ", "", "null", "NaN"]

# # Possible categorical values
RATING = [0,1,2,3,4]

df_train = pd.read_csv(training_data_uri)

# Remove NA values
def clean_dataframe(df):
    return df.replace(to_replace=NA_VALUES, value=np.NaN).dropna()

df_train = clean_dataframe(df_train)

user_ids = df_train["entity_type_users"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}

product_ids = df_train["product_id"].unique().tolist()
product2product_encoded = {x: i for i, x in enumerate(product_ids)}

df_train["user"] = df_train["entity_type_users"].map(user2user_encoded)
df_train["product"] = df_train["product_id"].map(product2product_encoded)
NUM_USERS = len(user2user_encoded)
NUM_PRODUCTS = len(product2product_encoded)


def convert_dataframe_to_dataset(
    df_train,
):
    NUMERIC_COLUMNS = ["entity_type_users","product_id","rating"]
    df_train[NUMERIC_COLUMNS] = df_train[NUMERIC_COLUMNS].astype("float32")
    df_train = df_train.drop(columns=UNUSED_COLUMNS)

    df_train_x, df_train_y = df_train, df_train.pop(LABEL_COLUMN)

    y_train = np.asarray(df_train_y).astype("float32")

    # Convert to numpy representation
    x_train = np.asarray(df_train_x)

    dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    return dataset_train

# Create datasets
dataset_train = convert_dataframe_to_dataset(df_train)

# Shuffle train set
dataset_train = dataset_train.shuffle(len(df_train))

EMBEDDING_SIZE = 50
class RecommenderNet(tf.keras.Model):
        def __init__(self, num_users, num_products, embedding_size, **kwargs):
            super(RecommenderNet, self).__init__(**kwargs)
            self.num_users = num_users
            self.num_products = num_products
            self.embedding_size = embedding_size
            self.user_embedding = tf.keras.layers.Embedding(
                num_users,
                embedding_size,
                embeddings_initializer="he_normal",
                embeddings_regularizer=tf.keras.regularizers.l2(1e-6),
            )
            self.user_bias = tf.keras.layers.Embedding(num_users, 1)
            self.product_embedding = tf.keras.layers.Embedding(
                num_products,
                embedding_size,
                embeddings_initializer="he_normal",
                embeddings_regularizer=tf.keras.regularizers.l2(1e-6),
            )
            self.product_bias = tf.keras.layers.Embedding(num_products, 1)

        def call(self, inputs):
            user_vector = self.user_embedding(inputs[:, 0])
            user_bias = self.user_bias(inputs[:, 0])
            product_vector = self.product_embedding(inputs[:, 1])
            product_bias = self.product_bias(inputs[:, 1])
            dot_user_product = tf.tensordot(user_vector, product_vector, 2)
            # Add all the components (including bias)
            x = dot_user_product + user_bias + product_bias
            # The sigmoid activation forces the rating to between 0 and 1
            return tf.nn.sigmoid(x)

def create_model(num_users,num_products):
    # Create model
        model = RecommenderNet(num_users, num_products, EMBEDDING_SIZE)
        model.compile(
            loss=tf.keras.losses.BinaryCrossentropy(),
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        )
        return model


model = create_model(num_users=NUM_USERS,num_products=NUM_PRODUCTS)

dataset_train = dataset_train.batch(args.batch_size)

# Train the model
model.fit(dataset_train, epochs=args.epochs)

tf.saved_model.save(model, os.getenv("AIP_MODEL_DIR"))

### 训练模型

使用`CustomTrainingJob`类来定义`TrainingPipeline`。该类接受以下参数：

- `display_name`：此训练流程的用户定义名称。
- `script_path`：训练脚本的本地路径。
- `container_uri`：训练容器镜像的URI。
- `requirements`：脚本的Python软件包依赖列表。
- `model_serving_container_image_uri`：可以为您的模型提供预测的容器的URI —— 可以是预构建的容器或自定义容器。

使用`run`函数开始训练。该函数接受以下参数：

- `args`：要传递给Python脚本的命令行参数。
- `replica_count`：worker副本的数量。
- `model_display_name`：如果脚本生成托管的`Model`，则为`Model`的显示名称。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要连接到worker副本的加速器数量。

`run`函数创建一个训练流程，训练并创建一个`Model`对象。训练流程完成后，`run`函数将返回`Model`对象。

In [ ]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DEPLOYED_NAME = f"product-recommender-{UUID}-{TIMESTAMP}"

job = aiplatform.CustomTrainingJob(
    display_name=DEPLOYED_NAME,
    script_path="task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["google-cloud-bigquery>=2.20.0", "db-dtypes"],
    model_serving_container_image_uri=DEPLOY_IMAGE,
)

# Start the training
model = job.run(
    model_display_name=DEPLOYED_NAME,
    args=CMDARGS,
    replica_count=1,
    machine_type=TRAIN_COMPUTE,
    accelerator_count=0,
)

### 部署模型

接下来，您可以将训练好的模型部署到`Endpoint`中。您可以通过在`Model`资源上调用`deploy`函数来实现这一点。这将执行两项操作：

1. 为部署`Model`资源创建一个`Endpoint`资源。
2. 将`Model`资源部署到`Endpoint`资源中。

该函数接受以下参数：

- `deployed_model_display_name`：部署模型的可读名称。
- `traffic_split`：在端点上发送到该模型的流量百分比，指定为一个或多个键/值对的字典。
   - 如果只有一个模型，那么请指定`{ "0": 100 }`，其中"0"指的是这个模型被上传，100表示100%的流量。
   - 如果端点上已有现有模型，流量将被拆分，则使用`model_id`指定`{ "0": 百分比, model_id: 百分比, ... }`，其中`model_id`是端点上现有`DeployedModel`的ID。百分比必须总和为100。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作实例的加速器数量。
- `starting_replica_count`：最初配置的计算实例数量。
- `max_replica_count`：要扩展到的最大计算实例数量。在本教程中，只配置一个实例。

#### 流量拆分

`traffic_split`参数指定为Python字典。您可以将模型的多个实例部署到一个端点，并设置流量比例分配到每个实例。

您可以使用流量拆分逐渐将新模型引入到生产环境中。例如，如果您已经在生产中拥有一个模型负责100%的流量，您可以部署一个新模型到同一个端点，将10%的流量引导到它，将原始模型的流量减少到90%。这样可以在最小化对大多数用户的干扰的同时监视新模型的性能。

#### 计算实例扩展

您可以指定单个实例（或节点）来提供您的在线预测请求服务。本教程使用单个节点，所以变量`MIN_NODES`和`MAX_NODES`都设置为`1`。

如果您想要使用多个节点来提供在线预测请求服务，请将`MAX_NODES`设置为您希望使用的节点的最大数量。Vertex AI会自动调整用于提供预测的节点数量，最多达到您设置的最大数量。请参考[定价页面](https://cloud.google.com/vertex-ai/pricing#prediction-prices)来了解使用多个节点进行自动缩放的成本。

#### Endpoint

该方法将阻塞直到模型部署完成，并最终返回一个`Endpoint`对象。如果这是第一次将模型部署到端点，则可能需要几分钟额外来完成资源的配置。

In [ ]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DEPLOYED_NAME = f"product-recommender-{UUID}-{TIMESTAMP}"

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

做一个预测
最后，您对部署到端点的推荐模型进行在线预测。

### 准备测试项目
您可以使用数据集的测试切片中的测试项目。

In [ ]:
import os

import numpy as np
import pandas as pd

# Set up training variables
LABEL_COLUMN = "rating"
UNUSED_COLUMNS = ["timestamp", "entity_type_users", "product_id"]
NA_VALUES = ["NA", ".", " ", "", "null", "NaN"]

# # Possible categorical values
RATING = [0, 1, 2, 3, 4]

df_test = pd.read_csv(CSV_FILE)


# Remove NA values
def clean_dataframe(df):
    return df.replace(to_replace=NA_VALUES, value=np.NaN).dropna()


df_test = clean_dataframe(df_test)

user_ids = df_test["entity_type_users"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
product_ids = df_test["product_id"].unique().tolist()
product_encoded2product = {i: x for i, x in enumerate(product_ids)}
product2product_encoded = {x: i for i, x in enumerate(product_ids)}

df_test["user"] = df_test["entity_type_users"].map(user2user_encoded)
df_test["product"] = df_test["product_id"].map(product2product_encoded)

sample = df_test.sample(1)
user_id = sample["user"].values[0]
products_bought = sample["product"].to_list()
products_not_bought = (
    df_test[~df_test["product"].isin(products_bought)]["product"].unique().tolist()
)

instances_input = [[float(user_id), k] for k in products_not_bought]

发送预测请求
接下来，您发送预测请求。

In [ ]:
prediction = endpoint.predict(instances=instances_input)
print(prediction)

获取前10个产品推荐
基于推荐模型预测的评分，我们为选定的`user_id`选择了前10个产品。

In [ ]:
predictions_array = np.array(
    [prediction.predictions[k][0] for k in range(len(prediction.predictions))]
)
top_rating_indices = predictions_array.argsort()[-10:][::-1]
top_predictions = predictions_array[top_rating_indices]
top_10_products = [
    int(product_encoded2product.get(instances_input[k][1])) for k in top_rating_indices
]
print(top_10_products)

清理工作
### 删除 BigQuery 数据集

使用方法 `delete_dataset()` 来删除一个 BigQuery 数据集以及其所有表，将参数 `delete_contents` 设置为 `True`。

In [ ]:
DESTINATION_DATASET = f"product_recommendation_{UUID}"
dataset_id = "{}.{}".format(PROJECT_ID, DESTINATION_DATASET)
dataset = bigquery.Dataset(dataset_id)
bqclient.delete_dataset(dataset, delete_contents=True)

### 删除 `Featurestore` 资源

您可以使用 `delete()` 方法删除指定的 `Featurestore` 资源，需要传入以下参数：

- `force`: 一个标志，指示是否删除非空的 `Featurestore` 资源。

In [ ]:
featurestore.delete(force=True)

删除Vertex AI `Model`和`Endpoint`

接下来，部署并删除Vertex AI Model和Endpoint资源。

In [ ]:
endpoint.undeploy_all()
endpoint.delete()
model.delete()

### 删除 Google Cloud 存储桶
最终，您删除了谷歌云存储桶

In [ ]:
! gsutil -m rm -r $BUCKET_URI
! gsutil rb $BUCKET_URI